# **Unraveling the Dynamics of Airfare Price Predictions**

Dennis Myasnyankin, Vannessa Salazar, and Christine Vu

Shiley-Marcos School of Engineering, University of San Diego

ADS 599: Capstone Project

Professor Ebrahim Tarshizi

December 11, 2023

***

### Import Packages

In [22]:
# Install library to load env file
    # %pip install python-dotenv

In [23]:
# Import requests and json libraries

import requests
import json
from dotenv import load_dotenv, dotenv_values

In [24]:
# Load in ENV information

config = dotenv_values(".env")

PCC = config['PCC']
COMPANY_NAME = config['COMPANY_NAME']
AUTH_TOKEN = config['AUTH_TOKEN']


### Define Functions

##### create_payload(origin, destination, timestamp)

In [25]:
# Function to create payload

def create_payload(origin, destination, timestamp):
    payload = {
        "OTA_AirLowFareSearchRQ": {
            "Target": "Production",
            "POS": {
                "Source": [
                    {
                        "PseudoCityCode": PCC,             # { ~ DOT_ENV ~ } #
                        "RequestorID": {
                            "Type": "1",
                            "ID": "1",
                            "CompanyName": {
                                "Code": COMPANY_NAME       # { ~ DOT_ENV ~ } #
                            }
                        }
                    }
                ]
            },
            "OriginDestinationInformation": [
                {
                    "RPH": "1",
                    "DepartureDateTime": timestamp,          # Date  =>  as Timestamp
                    "OriginLocation": {
                        "LocationCode": origin               # Origin =>  IATA Code
                    },
                    "DestinationLocation": {
                        "LocationCode": destination          # Destination => IATA Code
                    },
                    "TPA_Extensions": {
                        "SegmentType": {
                            "Code": "O"
                        },
                        "CabinPref": {
                            "PreferLevel": "Preferred",
                            "Cabin": "Y"                     # Cabin Class => API returns all available cabins
                        }
                    }
                }
            ],
            "TravelPreferences": {
                "MaxStopsQuantity": 0,
                "ValidInterlineTicket": True,
                "TPA_Extensions": {
                    "XOFares": {
                        "Value": True
                    },
                    "DataSources": {
                        "ATPCO": "Enable",
                        "LCC": "Enable",
                        "NDC": "Enable"
                    }
                }
            },
            "TravelerInfoSummary": {
                "SeatsRequested": [
                    1
                ],
                "AirTravelerAvail": [
                    {
                        "PassengerTypeQuantity": [
                            {
                                "Code": "ADT",
                                "Quantity": 1
                            }
                        ]
                    }
                ],
                "PriceRequestInformation": {
                    "CurrencyCode": "USD"
                }
            },
            "TPA_Extensions": {
                "IntelliSellTransaction": {
                    "RequestType": {
                        "Name": "100ITINS"                         
                    }
                },
                "ExchangeSettings": {
                    "AttachExchangeInfo": True
                }
            },
            "Version": "v4"
        }     
    }
    return payload

##### get_hub_jsons(tuple, timestamp)

In [26]:
# Function to extract flight route JSON

def get_hub_jsons(twople, timestamp):
    # Extract twople information
    hub = twople[0]
    destinations = twople[1]
    # Iterate destinations
    for dest in destinations:
        # Create payload, send POST request, extract JSON, save file to folder
        payload = create_payload(hub, dest, timestamp)
        r = requests.post(url, headers=headers, json=payload)
        content = r.json()
        f = open(f"airport_hubs/{hub}/{timestamp}/{dest}.json", 'w')
        f.write(json.dumps(content))
        f.close()

### Set Up HTTP POST Request Parameters

In [30]:
# Set Headers

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {AUTH_TOKEN}'     # { ~ DOT_ENV ~ } #
}

# Set URL

url = 'https://api.sabre.com/v4/offers/shop'


### Define Airport Hub - Destinations Tuple List

In [31]:
# Construct airport hub and destination tuples

hub_tuples = [
    ('DXB',	['RUH',	'KWI',	'BOM',	'JED',	'BAH']),
    ('LHR',	['JFK',	'DUB',	'AMS',	'FRA',	'MUC']),
    ('HKG',	['TPE',	'BKK',	'PVG',	'KIX',	'MNL']),
    ('ICN',	['KIX',	'FUK',	'NRT',	'TAO',	'HKG']),
    ('SIN',	['KUL',	'CKG',	'DPS',	'BKK',	'PEN']),
    ('CDG',	['LHR',	'AMS',	'BCN',	'JFK',	'FRA']),
    ('AMS',	['LHR',	'CPH',	'BCN',	'DUB',	'CDG']),
    ('FRA',	['LHR',	'BER',	'MUC',	'VIE',	'HAM']),
    ('BKK',	['CNX',	'HKT',	'USM',	'HKG',	'SIN']),
    ('IST',	['ADB',	'IKA',	'ESB',	'AYT',	'ADA']),
    ('KUL',	['SIN',	'CKG',	'BKI',	'KCH',	'PEN']),
    ('TPE',	['HKG',	'NRT',	'KIX',	'MNL',	'ICN']),
    ('DOH',	['BAH',	'AUH',	'KWI',	'LHR',	'SHJ']),
    ('MAD',	['BCN',	'LIS',	'PMI',	'FCO',	'LHR']),
    ('MUC',	['LHR',	'DUS',	'HAM',	'FRA',	'BER']),
    ('JFK',	['LHR',	'LAX',	'SFO',	'BOS',	'MIA']),
    ('PVG',	['SHE',	'DLC',	'CKG',	'CGQ',	'HKG']),
    ('BCN',	['PMI',	'MAD',	'FCO',	'AMS',	'LIS']),
    ('NRT',	['ICN',	'TPE',	'CTS',	'HKG',	'FUK']),
    ('FCO',	['CTA',	'MAD',	'BCN',	'PMO',	'LIN']),
    ('YYZ',	['LGA',	'YUL',	'YVR',	'ORD',	'YOW']),
    ('ZRH',	['LHR',	'AMS',	'FRA',	'BER',	'VIE']),
    ('VIE',	['FRA',	'LHR',	'ZRH',	'BER',	'AMS']),
    ('DUB',	['LHR',	'AMS',	'MAN',	'BHX',	'EDI']),
    ('PEK',	['SHA',	'SZX',	'CTU',	'HGH',	'CKG']),
    ('CPH',	['OSL',	'ARN',	'AMS',	'AAL',	'LHR']),
    ('LAX',	['JFK',	'SFO',	'LAS',	'PHX',	'DEN']),
    ('BRU',	['MAD',	'LHR',	'BCN',	'FRA',	'MUC']),
    ('LIS',	['MAD',	'ORY',	'BCN',	'FNC',	'LHR']),
    ('MIA',	['EWR',	'ATL',	'LGA',	'BOG',	'LAX'])
]

### Perform Data Extraction

In [9]:
# Iterate airport hubs and destinations for 1 day, 1 week and 1 month

for tup in hub_tuples:
    get_hub_jsons(tup, "2023-11-15T00:00:00")
    get_hub_jsons(tup, "2023-11-22T00:00:00")
    get_hub_jsons(tup, "2023-12-15T00:00:00")